In [1]:
%matplotlib inline
from fenparsev4 import *
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure.modules import TanhLayer
import numpy as np
import random
from sklearn.metrics import confusion_matrix
import os
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Position(board='         \n         \n rnbqkbnr\n pppp.ppp\n ........\n ....p...\n ....P...\n .....N..\n PPPP.PPP\n RNBQKB.R\n         \n         ', score=0, wc=(True, True), bc=(True, True), ep=0, kp=0)
(91, 42)
(81, 65)
(83, 63), P->.
Using Theano backend.


In [2]:
#most important part
def fries_ready():
    os.system('say your fries are done')

#returns dataframe with certain columns converted back to lists/integers
def unpickle_csv(filename_prefix='/media/drive/storage/csv_input/2015-12-08_112mil', filename_suffix_range=range(1,75), debug=True):
    #read in csv
    df = pd.DataFrame()
    for i in filename_suffix_range:
        if debug: print(filename_prefix + str(i))
        df = df.append(pd.read_csv(filename_prefix + str(i)))
    
    #clean columns
    df['y'] = df['y'].astype(int)
    if debug: print("converting x to list")
    df['x'] = df['x'] = df.loc[:, 'x'].apply(lambda x: [1 if '1' == a else 0 for a in x.split(', ')])
    length = df.shape[0]
    df = df.set_index([range(0,length)])
    return df;

df = unpickle_csv('/media/drive/storage/csv_input/2015-12-08_112mil', range(1,75))
fries_ready()
df.head()

/media/drive/storage/csv_input/2015-12-08_112mil1
/media/drive/storage/csv_input/2015-12-08_112mil2
/media/drive/storage/csv_input/2015-12-08_112mil3
/media/drive/storage/csv_input/2015-12-08_112mil4
/media/drive/storage/csv_input/2015-12-08_112mil5
/media/drive/storage/csv_input/2015-12-08_112mil6
/media/drive/storage/csv_input/2015-12-08_112mil7
/media/drive/storage/csv_input/2015-12-08_112mil8
/media/drive/storage/csv_input/2015-12-08_112mil9
/media/drive/storage/csv_input/2015-12-08_112mil10
/media/drive/storage/csv_input/2015-12-08_112mil11
/media/drive/storage/csv_input/2015-12-08_112mil12
/media/drive/storage/csv_input/2015-12-08_112mil13
/media/drive/storage/csv_input/2015-12-08_112mil14
/media/drive/storage/csv_input/2015-12-08_112mil15
/media/drive/storage/csv_input/2015-12-08_112mil16
/media/drive/storage/csv_input/2015-12-08_112mil17
/media/drive/storage/csv_input/2015-12-08_112mil18
/media/drive/storage/csv_input/2015-12-08_112mil19
/media/drive/storage/csv_input/2015-12-0

,Unnamed: 0,move,piece_moved,piece_captured,preboard,postboard,x,y
0,0,"(86, 76)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
1,0,"(92, 73)",n,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
2,0,"(87, 67)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
3,0,"(82, 62)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
4,0,"(87, 77)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1


In [3]:
#shuffle
df = df.reindex(np.random.permutation(df.index))

In [4]:
df.shape[0]

720509

In [5]:
#filtering for only white pawn moves
split = df.shape[0] * 4 / 5
all_train = df.iloc[:split, :]
all_test = df.iloc[split:, :]
# all_test = all_test[[s.piece_moved.islower() for s in all_test['move_info']]]
#all_test = all_test[[s.piece_moved.isupper() for s in all_test['move_info']]]
X_train = list(all_train['x'])
X_test = list(all_test['x'])
y_train = [[1] if y == 1 else [0] for y in all_train['y']]
y_test = [1 if y == 1 else 0 for y in all_test['y']]

In [17]:
Y_train = np_utils.to_categorical(y_train, 2)
Y_train.shape

(1287051, 2)

In [8]:
batch_size = 128
nb_classes = 2
nb_epoch = 5

X_train = np.array(X_train)
X_test = np.array(X_test)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

576407 train samples
144102 test samples


In [17]:
def getMLP(depth):
    if depth < 3:
        depth = 3
    model = Sequential()
    model.add(Dense(512, input_shape=(1536,)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2))
    for i in range(0, depth - 2):
        model.add(Dense(512))
        model.add(Activation('tanh'))
        model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    rms = RMSprop()
    model.compile(loss='categorical_crossentropy', optimizer=rms)
    #print(model.to_json())

    return model

In [16]:
model = getMLP(5)

adding layer
adding layer
adding layer
{"layers": [{"b_constraint": null, "name": "Dense", "activity_regularizer": null, "W_constraint": null, "input_shape": [1536], "init": "glorot_uniform", "activation": "linear", "input_dim": null, "b_regularizer": null, "W_regularizer": null, "output_dim": 512}, {"beta": 0.1, "activation": "tanh", "name": "Activation", "target": 0}, {"p": 0.2, "name": "Dropout"}, {"b_constraint": null, "name": "Dense", "activity_regularizer": null, "W_constraint": null, "init": "glorot_uniform", "activation": "linear", "input_dim": null, "b_regularizer": null, "W_regularizer": null, "output_dim": 512}, {"beta": 0.1, "activation": "tanh", "name": "Activation", "target": 0}, {"p": 0.2, "name": "Dropout"}, {"b_constraint": null, "name": "Dense", "activity_regularizer": null, "W_constraint": null, "init": "glorot_uniform", "activation": "linear", "input_dim": null, "b_regularizer": null, "W_regularizer": null, "output_dim": 512}, {"beta": 0.1, "activation": "tanh", "na

In [ ]:
# print(type(X_train))
# print(Y_train.shape)
# print(Y_test.shape) 
# sys.exit()
model = getMLP(5)
model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test,
                       show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

adding layer
adding layer
adding layer
Train on 576407 samples, validate on 144102 samples

In [11]:
all_train.head()

,Unnamed: 0,move,piece_moved,piece_captured,preboard,postboard,x,y
294202,0,"(96, 52)",B,.,"[' rnbqkbnr\n', ' pppppppp\n', ' ........\n', ...","([' rnbqkbnr\n', ' pppppppp\n', ' ........\n',...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
574443,0,"(73, 51)",B,.,"[' r....rk.\n', ' pppb..pp\n', ' ..nqp...\n', ...","([' r....rk.\n', ' pppb..pp\n', ' ..nqp...\n',...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
53737,0,"(82, 46)",B,p,"[' ....r.k.\n', ' pp....p.\n', ' .....p.p\n', ...","([' ....r.k.\n', ' pp....p.\n', ' .....p.p\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
296214,0,"(82, 72)",p,.,"[' .KR.Q..R\n', ' ..P...P.\n', ' P.N.....\n', ...","([' .KR.Q..R\n', ' ..P...P.\n', ' P.N.....\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
333190,0,"(88, 68)",P,.,"[' ...qkb.r\n', ' ..r.pppp\n', ' B.p..n..\n', ...","([' ...qkb.r\n', ' ..r.pppp\n', ' B.p..n..\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [51]:
class Experiment:
    def __init__(self, net, train_df, test_df, num_epochs, verbose=True):
        self.net = net
        self.train_df = train_df
        self.test_df = test_df
        self.train_ds = self.build_dataset(train_df)
        self.test_ds = self.build_dataset(test_df)
        self.num_epochs = num_epochs
        self.verbose = verbose
        
    #takes in full dataframe and converts to pybrain dataset
    def build_dataset(self, df):
        ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
        for i in df.index:
            ds.addSample(df.loc[i,'x'], df.loc[i,'y'])
        ds._convertToOneOfMany(bounds=[0, 1])
        return ds

    #adds specific piece confusion matrices to results dict d
    def add_piece_specifics(d):
        pass
        
    
    def run_experiment(self):
           
        trainer = BackpropTrainer(self.net,  self.train_ds, verbose=self.verbose)
        print "Built network"
        y_pred = []

        trainer.trainEpochs(self.num_epochs)
        prob_out = net.activateOnDataset(self.test_ds)
        y_pred = prob_out.argmax(axis=1) 
        
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
    #    print(confusion_matrix(y_train, out))
        return pd.DataFrame({"train_size": self.train_df.shape[0], 
                            "train_white_count" : sum([1 if a.isupper() else 0 for a in self.train_df['piece_moved']]),
                            "confusion_matrix" : [cm],
                            "accuracy": [(cm[0][0] + cm[1][1]) * 1.0 / (sum([sum(c) for c in cm]))]})

In [52]:
class Experiment2:
    #takes in full dataframe and converts to pybrain dataset
    def build_dataset(self, df):
        ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
        for i in df.index:
            ds.addSample(df.loc[i,'x'], df.loc[i,'y'])
        ds._convertToOneOfMany(bounds=[0, 1])
        return ds

    #adds specific piece confusion matrices to results dict d
    def add_piece_specifics(d):
        pass
        
    
    def run_experiment(self, train_all, test_all, num_epochs):
        train_ds = self.build_dataset(train_all)
        test_ds = self.build_dataset(test_all)

        trainer = BackpropTrainer(net, train_ds, verbose=True)
        print "Built network"
        y_pred = []

        trainer.trainEpochs(num_epochs)
        prob_out = net.activateOnDataset(test_ds)
        y_pred = prob_out.argmax(axis=1)  # the highest output activation gives the class
        #out = out.reshape(X_train.shape)
        #print(y_pred)
        print(prob_out)
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
    #    print(confusion_matrix(y_train, out))
        return pd.DataFrame({"train_size": train_all.shape[0], \
                            "train_white_count" : sum([1 if a.isupper() else 0 for a in train_all['piece_moved']]),\
                            "confusion_matrix" : [cm],
                            "accuracy": [(cm[0][0] + cm[1][1]) * 1.0 / (sum([sum(c) for c in cm]))]})

In [44]:
from pybrain.structure import FeedForwardNetwork, RecurrentNetwork
from pybrain.structure import LinearLayer, SigmoidLayer
from pybrain.structure import FullConnection

def buildSimpleNet(hidden_layers, output_layers, bias, hidden_class):
    return buildNetwork(1536, hidden_layers, output_layers, bias, hiddenclass=hidden_class)

def buildFeedForward():
    n = FeedForwardNetwork()
    
    inLayer = LinearLayer(1536)
    hiddenLayer = SigmoidLayer(3)
    outLayer = LinearLayer(2)
    
    n.addInputModule(inLayer)
    n.addModule(hiddenLayer)
    n.addOutputModule(outLayer)
    
    in_to_hidden = FullConnection(inLayer, hiddenLayer)
    hidden_to_out = FullConnection(hiddenLayer, outLayer)
    
    n.addConnection(in_to_hidden)
    n.addConnection(hidden_to_out)
    
    n.sortModules()
    return n
def buildBigFeedForward():
    n = FeedForwardNetwork()
    
    inLayer = LinearLayer(1536)
    hiddenLayer = SigmoidLayer(100)
    hiddenLayer2 = LinearLayer(50)
    hiddenLayer3 = SigmoidLayer(10)
    outLayer = LinearLayer(2)
    
    n.addInputModule(inLayer)
    n.addModule(hiddenLayer)
    n.addModule(hiddenLayer2)
    n.addModule(hiddenLayer3)
    n.addOutputModule(outLayer)
    
    in_to_hidden = FullConnection(inLayer, hiddenLayer)
    hidden_to_hidden2 = FullConnection(hiddenLayer, hiddenLayer2)
    hidden2_to_hidden3 = FullConnection(hiddenLayer, hiddenLayer3)

    hidden_to_out = FullConnection(hiddenLayer3, outLayer)
    
    n.addConnection(in_to_hidden)
    n.addConnection(hidden_to_hidden2)
    n.addConnection(hidden2_to_hidden3)

    n.addConnection(hidden_to_out)
    
    n.sortModules()
    return n
def buildRNN():
    n = RecurrentNetwork()
    
    n.addInputModule(LinearLayer(1536, name='in'))
    n.addModule(SigmoidLayer(3, name='hidden'))
    n.addOutputModule(LinearLayer(2, name='out'))
    
    n.addConnection(FullConnection(n['in'], n['hidden'], name='c1'))
    n.addConnection(FullConnection(n['hidden'], n['out'], name='c2'))
    
    n.addRecurrentConnection(FullConnection(n['hidden'], n['hidden'], name='c3'))
   
    n.sortModules()
    
    return n

In [47]:
# results_df = pd.DataFrame()
# for train_instances in [10, 500, 1000, 2000, 3000, 4000, 5000]:
#     e = Experiment()
#     results_df = results_df.append(e.run_experiment(all_train[:train_instances], all_test, 1))
net  = buildNetwork(1536, 3, 2, bias=True, hiddenclass=TanhLayer)
#net = buildSimpleNet(2, 2, True, TanhLayer)
#net = buildFeedForward()
#net = buildRNN()
#net = buildBigFeedForward()
e = Experiment(net, all_train, all_test, 5)
results_df = e.run_experiment()
fries_ready()
#e = Experiment2()
#results_df = e.run_experiment(all_train, all_test, 1)

Built network
Total error: 0.137738591832
Total error: 0.127366845835
Total error: 0.127452412146
Total error: 0.127371025199
Total error: 0.127392883597
[[674   3]
 [695   0]]


In [48]:
results_df

,accuracy,confusion_matrix,train_size,train_white_count
0,0.491254,"[[674, 3], [695, 0]]",24584,24584


In [30]:
#[1 if a.isupper() else 0 for a in all_train['piece_moved']]
df.head()

,move,piece_moved,piece_captured,preboard,postboard,x,y
9690,"(88, 68)",p,.,"[ RNBKQBNR\n, PPP.PPPP\n, ........\n, ...P....","([ RNBKQBNR\n, PPP.PPPP\n, ........\n, ...P...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0
402,"(71, 81)",R,.,"[ r.....k.\n, pprn.p.q\n, ..p.bBp.\n, .PPpP...","([ r.....k.\n, pp.n.prq\n, ..p.bBp.\n, .PPp...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
6565,"(86, 85)",q,.,"[ .K.R.BNR\n, PPP.P.PP\n, ........\n, ........","([ .K.R.BNR\n, PPP.P.PP\n, ........\n, .......","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
915,"(94, 95)",R,.,"[ .....rk.\n, .....pp.\n, ..p.q.np\n, .pQ.P...","([ .....rk.\n, .....pp.\n, ..p.q.np\n, .pQ....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2743,"(75, 76)",R,.,"[ ..r...k.\n, .p.R..bp\n, r...p.p.\n, p...P...","([ ......k.\n, .p.R..bp\n, r...p.p.\n, p......","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [9]:
#train a simple neural network on 2000 instances
print "Starting"
# build dataset
train_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
for i in range(0,500):
    #j = random.randint(0,2000)
    train_ds.addSample(X_train[i], y_train[i])
    
test_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
for i in range(0,500):
    test_ds.addSample(X_test[i], y_test[i])
print "Building dataset"
#print(train_ds.getField('target'))
train_ds._convertToOneOfMany(bounds=[0, 1])
test_ds._convertToOneOfMany(bounds=[0, 1])
#print(train_ds.getField('target'))
print "Converted"
net     = buildNetwork(1536, 100, 2, bias=True, hiddenclass=TanhLayer)
trainer = BackpropTrainer(net,  train_ds, verbose=True)
print "Built network"
y_pred = []
for i in range(1):
    trainer.trainEpochs( 1 )
    prob_out = net.activateOnDataset(test_ds)
    y_pred = prob_out.argmax(axis=1)  # the highest output activation gives the class
    #out = out.reshape(X_train.shape)
    print(y_pred)
    #print(prob_out)
    print(confusion_matrix(y_test, y_pred))
#    print(confusion_matrix(y_train, out))


Starting
Building dataset
Converted
Built network
Total error: 8.2274069558
[1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1
 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1
 0 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1
 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1
 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0
 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0
 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0
 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1
 0 0 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1
 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0
 0 0 1 0 0 1 1 1 0 0 1 1

In [10]:
y_true = pd.Series(y_test)
y_pred = pd.Series(y_pred)
pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,125,119,244
1,119,137,256
All,244,256,500


In [11]:
y_pred = pd.Series(y_pred)
length = all_test.shape[0]
all_test = all_test.set_index([range(0,length)])
all_test.loc[:,'predicted'] = y_pred

In [12]:
y_pred.head()

0    1
1    1
2    0
3    0
4    1
dtype: int64

In [13]:
all_test = all_test.iloc[:500,:]

In [15]:
def collect_results(test_df, predicted_y)
    result_df = pd.DataFrame(columns=['training_instances', 'testingpct_white_moves',])
    
    #add predicted
    length = all_test.shape[0]
    test_df = all_test.set_index([range(0,length)])
    test_df.loc[:,'predicted'] = y_pred
    
    #calculate overall confusion matrix
    cm = confusion_matrx(test_df['y'], predicted_y)
    
    #calculate each piece confusion matrix
    for p in "pPrRnNbBqQkK":
        specific_piece = all_test[all_test['piece_moved'] == p]
#         print(p)
        cm = confusion_matrix(specific_piece['y'], specific_piece['predicted'])
        test_df.loc[:, p + '_perf'] = cm
#         print(cm)
#         print(1.0 * cm[0][0] / (sum([sum(a) for a in cm])))

p
[[36 20]
 [29 16]]
0.356435643564
P
[[13 23]
 [16 23]]
0.173333333333
r
[[ 5  7]
 [ 8 10]]
0.166666666667
R
[[14  6]
 [12 12]]
0.318181818182
n
[[17  4]
 [11  6]]
0.447368421053
N
[[ 4 11]
 [ 4 12]]
0.129032258065
b
[[6 3]
 [8 8]]
0.24
B
[[ 5 11]
 [ 2 16]]
0.147058823529
q
[[11 10]
 [ 6 11]]
0.289473684211
Q
[[ 9  8]
 [ 7 10]]
0.264705882353
k
[[2 9]
 [8 6]]
0.08
K
[[3 7]
 [8 7]]
0.12
